In [23]:
from typing import Optional, Tuple, Callable, Dict
import math
import string

import math
from typing import Tuple, List

import math
from typing import Tuple


In [24]:

def bits_to_pauli(x_bits: int, z_bits: int, n_qubits: int) -> str:
    pauli_map = 'IZXY'
    chars = []
    for i in range(n_qubits - 1, -1, -1):
        x_i = (x_bits >> i) & 1
        z_i = (z_bits >> i) & 1
        map_index = (x_i << 1) | z_i
        chars.append(pauli_map[map_index])
    return ''.join(chars)

def pauli_to_bits(pauli: str) -> Tuple[int, int]:
    tx = str.maketrans({'I': '0', 'Z': '0', 'X': '1', 'Y': '1'})
    tz = str.maketrans({'I': '0', 'X': '0', 'Z': '1', 'Y': '1'})
    bx_be = pauli.translate(tx)
    bz_be = pauli.translate(tz)
    return int(bx_be, 2), int(bz_be, 2)

In [25]:
class PauliTerm:
    """
    Represents a single Pauli term with a coefficient and bit masks for X and Z.
    
    Attributes:
        coeff (complex): Coefficient of the Pauli term.
        x_bits (int): Bit mask indicating positions of X.
        z_bits (int): Bit mask indicating positions of Z.
        n_qubits (int): Number of qubits (length of the Pauli string).
    """

    bits_to_pauli = staticmethod(bits_to_pauli)
    
    def __init__(self, coeff: complex, x_bits: int, z_bits: int, n_qubits: int):
        self.coeff = coeff
        self.x_bits = x_bits
        self.z_bits = z_bits
        self.n_qubits = n_qubits
    
    def __repr__(self) -> str:
        pauli_str = self.bits_to_pauli(self.x_bits, self.z_bits, self.n_qubits)
        coeff_str = f"{self.coeff:+g}"
        return f"{coeff_str} {pauli_str}"


In [ ]:
class QuantumGate:
    """
    A registry for single‐qubit conjugation rules.
    Use @QuantumGate.register("gate_name") to register a function
    that takes (PauliTerm, qubit_index) and returns List[PauliTerm].
    """
    _registry: Dict[str, Callable] = {}

    @classmethod
    def register(cls, name: str) -> Callable:
        """
        Decorator to register a new gate handler under the given name.
        Usage:
            @QuantumGate.register("t")
            def t_on_qubit_term(...):
                ...
        """
        def decorator(fn: Callable) -> Callable:
            cls._registry[name] = fn
            return fn
        return decorator

    @classmethod
    def get(cls, name: str) -> Callable:
        """
        Retrieve the registered handler for gate `name`.
        Raises NotImplementedError if not found.
        """
        if name not in cls._registry:
            raise NotImplementedError(f"No rule for gate '{name}'")
        return cls._registry[name]



@QuantumGate.register("t")
def t_on_qubit_term(term: PauliTerm, q: int) -> List[PauliTerm]:
    """
    Conjugate a PauliTerm by T on qubit q:  T† P T
    Returns a list of 1 or 2 PauliTerm.
    """
    # Pack x_bits and z_bits into a single integer key (lower n_qubits bits are x_bits, higher n_qubits bits are z_bits)
    key = term.x_bits | (term.z_bits << term.n_qubits)
    n = term.n_qubits

    # Extract x_q and z_q for this qubit from key
    x_q = (key >> q)       & 1
    z_q = (key >> (n + q)) & 1

    if x_q == 0: # For I or Z (x_q=0), return the original term without splitting
        return [ PauliTerm(term.coeff, term.x_bits, term.z_bits, n) ]

    # For X or Y (x_q=1), split into two terms
    # First term keeps original key, corresponding to original P component
    c1 = term.coeff / math.sqrt(2)
    k1 = key

    # Second term flips Z_q
    # T† X T = (X + Y)/√2  → c2 = +c1 when z_q=0
    # T† Y T = (Y − X)/√2  → c2 = −c1 when z_q=1
    c2 = (-c1) if (z_q == 0) else (c1)
    k2 = key ^ (1 << (n + q))

    # Unpack into two PauliTerm
    mask = (1 << n) - 1
    x1, z1 = k1 & mask, k1 >> n
    x2, z2 = k2 & mask, k2 >> n

    return [PauliTerm(c1, x1, z1, n), PauliTerm(c2, x2, z2, n)]


@QuantumGate.register("cx")
def cx_on_qubit_term(term: PauliTerm, c: int, t: int) -> List[PauliTerm]:
    """
    Conjugate a PauliTerm by CX (control=c, target=t) in Heisenberg picture:
      X_c ⟼ X_c ⊗ X_t
      Z_t ⟼ Z_c ⊗ Z_t
      all other Paulis unchanged
    Returns a list of exactly one PauliTerm.
    """
    n = term.n_qubits
    # pack x_bits and z_bits into key
    key = term.x_bits | (term.z_bits << n)

    # extract the four relevant bits
    x_c = (key >> c      ) & 1
    z_c = (key >> (n + c)) & 1
    x_t = (key >> t      ) & 1
    z_t = (key >> (n + t)) & 1

    # apply conjugation rules
    
    if x_c: # if X_c is present, then X_t must also appear
        x_t ^= 1
    if z_t: # if Z_t is present, then Z_c must also appear
        z_c ^= 1

    # rebuild key
    key = (key & ~(1 << c))      | (x_c << c) # set control X bit
    key = (key & ~(1 << t))      | (x_t << t) # set target X bit
    key = (key & ~(1 << (n + c))) | (z_c << (n + c)) # set control Z bit
    key = (key & ~(1 << (n + t))) | (z_t << (n + t)) # set target Z bit

    # unpack back into x_bits, z_bits
    mask = (1 << n) - 1
    x_new = key & mask
    z_new = key >> n

    return [ PauliTerm(term.coeff, x_new, z_new, n) ]

